In [1]:
import pandas as pd
import numpy as np
from pathlib import Path
import csv
import xlrd
import io
from zipfile import ZipFile
from datetime import date, timedelta, time, datetime

In [2]:
today_yyyymmdd = date.today().strftime("%Y%m%d")
print(today_yyyymmdd)

20200825


import datetime 
base = datetime.date(2020,2,1)
for i in range(0,10): 
    d=base + datetime.timedelta(i)
    d=d.strftime("%Y%m%d")
    print(f"N029kakutokuruiseki{d}.csv")

from datetime import datetime, timedelta

def date_range(start_date: datetime, end_date: datetime):
    diff = (end_date - start_date).days + 1
    return (start_date + timedelta(i) for i in range(diff))

In [3]:
#pandasの表示列数を増やす
import pandas as pd
pd.set_option('display.max_columns', 100)

ディレクトリ

In [4]:
data_dir = Path(r"C:\Users\crie072\Documents\21_Online")

Inputファイル設定

In [5]:
#承認月
approval_month = "8月"

#①承認リスト（当月分）
input_file_1 = "N029（2020年8月1～24日入会）.xlsx"
#②承認リスト（前月分）
input_file_2 = "N029（2020年7月入会）.xlsx"
#②承認リスト（前々月）追加
input_file_4 = "N029（2020年6月入会）.xlsx"

#③ デジガレ確認用リスト
input_file_3 = "20200821〆【JFRカード御中】成果データ&問い合わせ.xlsx"

Outputファイル定義

In [6]:
#Output_file
#①オンライン承認リスト
output_file_1 = f"当月オンライン承認リスト_{today_yyyymmdd}" 
print("Output①⇒",output_file_1)

#②アフィリエイト月次成果承認確定リスト
output_file_2 = f"デジガレ成果判定データ_{today_yyyymmdd}" 
print("Output②⇒",output_file_2)

Output①⇒ 当月オンライン承認リスト_20200825
Output②⇒ デジガレ成果判定データ_20200825


①承認リスト（当月分）を読み込む

In [7]:
print(input_file_1)

df_app = pd.read_excel(data_dir /r"input"/f"{input_file_1}")
df_app.head()

N029（2020年8月1～24日入会）.xlsx


,カード加入日,会員カード発行区分,続柄,カード提携会社1CD,カード提携会社2CD,カード種類CD,カードブランドCD,切替CD,勧誘CD,受付部支店,性別CD,再発行CD,旧カード提携会社1CD,旧カード提携会社2CD,旧カード種類CD,旧カードブランドCD,入会申込番号,満年齢,リボ支払方法CD,勧誘団体CD,提携カード特殊エリア,エンボス内容,カード年会費CD,リボ利用対象CD
0,20200801,D,0,D00,50,31,0,0,8,101,1,0,NaN,NaN,NaN,NaN,155203037822361,41,12.0,1000,20.0,1.000000e+03,1,0
1,20200801,D,0,D00,50,31,0,0,8,107,2,0,NaN,NaN,NaN,NaN,155203037822080,40,12.0,7000,20.0,7.000000e+03,1,0
2,20200801,D,0,D00,50,31,0,0,8,107,2,0,NaN,NaN,NaN,NaN,155203047822625,40,12.0,7000,20.0,7.000404e+09,1,0
3,20200801,D,0,D00,50,31,0,0,8,107,1,0,NaN,NaN,NaN,NaN,155203037823815,55,12.0,7000,20.0,7.000534e+09,1,0
4,20200801,D,0,D00,50,31,0,0,8,107,2,0,NaN,NaN,NaN,NaN,155203037822197,48,12.0,7000,20.0,7.000401e+09,1,0


In [8]:
print(df_app.shape)
print(df_app.index)
print(df_app.columns)
print(df_app.dtypes)

(4975, 24)
RangeIndex(start=0, stop=4975, step=1)
Index(['カード加入日', '会員カード発行区分', '続柄', 'カード提携会社1CD', 'カード提携会社2CD', 'カード種類CD',
       'カードブランドCD', '切替CD', '勧誘CD', '受付部支店', '性別CD', '再発行CD', '旧カード提携会社1CD',
       '旧カード提携会社2CD', '旧カード種類CD', '旧カードブランドCD', '入会申込番号', '満年齢', 'リボ支払方法CD',
       '勧誘団体CD', '提携カード特殊エリア', 'エンボス内容', 'カード年会費CD', 'リボ利用対象CD'],
      dtype='object')
カード加入日           int64
会員カード発行区分       object
続柄               int64
カード提携会社1CD      object
カード提携会社2CD       int64
カード種類CD          int64
カードブランドCD        int64
切替CD             int64
勧誘CD             int64
受付部支店            int64
性別CD             int64
再発行CD            int64
旧カード提携会社1CD     object
旧カード提携会社2CD    float64
旧カード種類CD       float64
旧カードブランドCD     float64
入会申込番号           int64
満年齢              int64
リボ支払方法CD       float64
勧誘団体CD           int64
提携カード特殊エリア     float64
エンボス内容         float64
カード年会費CD         int64
リボ利用対象CD         int64
dtype: object


オンライン入会客を特定

In [9]:
df_app =df_app[(df_app["会員カード発行区分"] == "1") 
               & (df_app["続柄"] == 0) 
               & ~(df_app["カード提携会社2CD"] == 95) 
               & (df_app["切替CD"] == 0) 
               & (df_app["受付部支店"] == 299)]
#含まれてないか確認
print(df_app["会員カード発行区分"].unique())
print(df_app["続柄"].unique())
print(df_app["カード提携会社2CD"].unique())
print(df_app["切替CD"].unique())
print(df_app["受付部支店"].unique())
df_app.shape[0]

['1']
[0]
[ 4  1  8 14 18 11]
[0]
[299]


547

In [10]:
#"成果"カラムを追加
df_app['成果'] = "可"
df_app['参照用'] = df_app['入会申込番号'] #"参照用"はデジガレとのマッチング用にリネーム

df_app=df_app[['入会申込番号','成果','カード加入日','参照用']]
df_app.head()

,入会申込番号,成果,カード加入日,参照用
59,155051921979437,可,20200803,155051921979437
60,155051921916363,可,20200803,155051921916363
63,155051921955510,可,20200803,155051921955510
64,155051921978223,可,20200803,155051921978223
72,155051921969560,可,20200803,155051921969560


CSVアウトプット①

In [11]:
df_tmp = df_app.drop(columns=['参照用'])
df_tmp.to_csv(data_dir /r"output"/f"{output_file_1}.csv", index=False,header=True,encoding='shift-jis')

②前月の承認結果作業を読み込む

In [12]:
print(input_file_2)

df_pre = pd.read_excel(data_dir /r"input"/f"{input_file_2}")
df_pre.head()

N029（2020年7月入会）.xlsx


,カード加入日,会員カード発行区分,続柄,カード提携会社1CD,カード提携会社2CD,カード種類CD,カードブランドCD,切替CD,勧誘CD,受付部支店,性別CD,再発行CD,旧カード提携会社1CD,旧カード提携会社2CD,旧カード種類CD,旧カードブランドCD,入会申込番号,満年齢,リボ支払方法CD,勧誘団体CD,提携カード特殊エリア,エンボス内容,カード年会費CD,リボ利用対象CD
0,20200701,1,0,D00,8,31,1,0,3,299,2,0,NaN,NaN,NaN,NaN,155051921910960,46,12.0,7000,40.0,7.000000e+03,2,0
1,20200701,1,0,D00,1,31,1,0,3,299,1,0,NaN,NaN,NaN,NaN,155051921918211,50,52.0,6000,40.0,6.000000e+03,2,1
2,20200701,2,1,D00,8,31,1,0,3,1,1,0,NaN,NaN,NaN,NaN,155051921944480,58,52.0,2000,NaN,2.000310e+09,0,1
3,20200701,1,0,D02,1,2,1,0,3,102,1,0,NaN,NaN,NaN,NaN,155603602468245,36,12.0,2000,NaN,2.000010e+09,2,0
4,20200701,5,0,D00,95,0,0,0,3,1,2,0,NaN,NaN,NaN,NaN,155603602537692,47,NaN,0,NaN,NaN,1,0


オンライン入会客を特定

In [13]:
df_pre =df_pre[(df_pre["会員カード発行区分"] == "1") 
               & (df_pre["続柄"] == 0) 
               & ~(df_pre["カード提携会社2CD"] == 95) 
               & (df_pre["切替CD"] == 0) 
               & (df_pre["受付部支店"] == 299)]
#含まれてないか確認
print(df_pre["会員カード発行区分"].unique())
print(df_pre["続柄"].unique())
print(df_pre["カード提携会社2CD"].unique())
print(df_pre["切替CD"].unique())
print(df_pre["受付部支店"].unique())
df_pre.shape[0]

['1']
[0]
[ 8  1  4 18 14 11]
[0]
[299]


889

In [14]:
#"成果"カラムを追加
df_pre['成果'] = "可"
df_pre['参照用'] = df_pre['入会申込番号'] #"参照用"はデジガレとのマッチング用にリネーム

df_pre=df_pre[['入会申込番号','成果','カード加入日','参照用']]
df_pre.head()

,入会申込番号,成果,カード加入日,参照用
0,155051921910960,可,20200701,155051921910960
1,155051921918211,可,20200701,155051921918211
5,155051921922544,可,20200701,155051921922544
11,155051921940074,可,20200701,155051921940074
31,155051921944332,可,20200701,155051921944332


前月分と当月分の承認リストを縦結合

In [15]:
df_app = pd.concat([df_pre, df_app])
df_app.shape[0]

1436

In [16]:
print(df_app['カード加入日'].unique())

[20200701 20200702 20200703 20200706 20200707 20200708 20200709 20200710
 20200713 20200714 20200715 20200716 20200717 20200720 20200721 20200722
 20200727 20200728 20200729 20200730 20200731 20200803 20200804 20200805
 20200806 20200807 20200811 20200812 20200813 20200814 20200817 20200818
 20200819 20200820 20200821 20200824]


②-2前々月の承認結果作業を読み込む　追加対応

In [17]:
print(input_file_4)

df_pre = pd.read_excel(data_dir /r"input"/f"{input_file_4}")
df_pre.head()

N029（2020年6月入会）.xlsx


,カード加入日,会員カード発行区分,続柄,カード提携会社1CD,カード提携会社2CD,カード種類CD,カードブランドCD,切替CD,勧誘CD,受付部支店,性別CD,再発行CD,旧カード提携会社1CD,旧カード提携会社2CD,旧カード種類CD,旧カードブランドCD,入会申込番号,満年齢,リボ支払方法CD,勧誘団体CD,提携カード特殊エリア,エンボス内容,カード年会費CD,リボ利用対象CD
0,20200601,1,0,D00,1,31,1,0,3,299,1,0,NaN,NaN,NaN,NaN,155051921903874,37,12.0,2000,40.0,2.000000e+03,2,0
1,20200601,1,0,D02,4,2,2,0,3,102,1,0,NaN,NaN,NaN,NaN,155603602336764,52,12.0,2000,NaN,2.000010e+09,2,0
2,20200601,1,0,D02,4,2,2,0,3,102,1,0,NaN,NaN,NaN,NaN,155603602336533,43,12.0,2000,NaN,2.000010e+09,2,0
3,20200601,1,1,D02,1,2,1,0,3,102,2,0,NaN,NaN,NaN,NaN,155603602336251,48,12.0,2000,NaN,2.000010e+09,2,0
4,20200601,D,0,D00,50,31,0,0,8,103,2,0,NaN,NaN,NaN,NaN,155203047717981,71,12.0,3000,20.0,3.000079e+09,1,0


オンライン入会客を特定

In [18]:
df_pre =df_pre[(df_pre["会員カード発行区分"] == "1") 
               & (df_pre["続柄"] == 0) 
               & ~(df_pre["カード提携会社2CD"] == 95) 
               & (df_pre["切替CD"] == 0) 
               & (df_pre["受付部支店"] == 299)]
#含まれてないか確認
print(df_pre["会員カード発行区分"].unique())
print(df_pre["続柄"].unique())
print(df_pre["カード提携会社2CD"].unique())
print(df_pre["切替CD"].unique())
print(df_pre["受付部支店"].unique())
df_pre.shape[0]

['1']
[0]
[ 1  8  4 11 14]
[0]
[299]


697

In [19]:
#"成果"カラムを追加
df_pre['成果'] = "可"
df_pre['参照用'] = df_pre['入会申込番号'] #"参照用"はデジガレとのマッチング用にリネーム

df_pre=df_pre[['入会申込番号','成果','カード加入日','参照用']]
df_pre.head()

,入会申込番号,成果,カード加入日,参照用
0,155051921903874,可,20200601,155051921903874
19,155051921903460,可,20200601,155051921903460
35,155051921902660,可,20200601,155051921902660
38,155051921902561,可,20200601,155051921902561
47,155051921878019,可,20200601,155051921878019


前々月分の承認リストを縦結合

In [20]:
df_app = pd.concat([df_pre, df_app])
df_app.shape[0]

2133

In [21]:
print(df_app['カード加入日'].unique())

[20200601 20200602 20200603 20200604 20200605 20200608 20200609 20200610
 20200611 20200612 20200615 20200616 20200617 20200618 20200619 20200622
 20200623 20200624 20200625 20200626 20200629 20200630 20200701 20200702
 20200703 20200706 20200707 20200708 20200709 20200710 20200713 20200714
 20200715 20200716 20200717 20200720 20200721 20200722 20200727 20200728
 20200729 20200730 20200731 20200803 20200804 20200805 20200806 20200807
 20200811 20200812 20200813 20200814 20200817 20200818 20200819 20200820
 20200821 20200824]


③ デジガレからの成果確認依頼リスト

In [22]:
print(input_file_3)

df_dg = pd.read_excel(data_dir/r"input"/f"{input_file_3}", sheet_name = 0)

#シート名取得
bk = pd.ExcelFile(data_dir/r"input"/f"{input_file_3}")
print(bk.sheet_names)

df_dg.head()

20200821〆【JFRカード御中】成果データ&問い合わせ.xlsx
['成果データ', '識別子不明', '成果データ（5月～利用条件追加媒体）', '成果データ（5月～利用条件追加媒体）_承認返却済み']


,ASID,メディア名,媒体チャネル,クリック,申込,申込月,参照用,ASP名,単価,成果,カード発行日
0,8163.0,POM[ポム],ポイント,2020-04-23 10:31:22,2020-04-23 10:43:06,43922,155051921858227,ADV,5000,保留,保留
1,8163.0,POM[ポム],ポイント,2020-04-24 10:17:31,2020-04-24 10:27:05,43922,155051921859886,ADV,5000,保留,保留
2,37.0,げん玉,ポイント,2020-04-26 20:21:03,2020-04-26 20:29:07,43922,155051921864571,BGT,5000,保留,保留
3,1580002.0,クレジットカード忍法帖,比較,2020-04-27 13:43:00,2020-04-27 14:31:00,43922,155051921865248,JA,9000,保留,保留
4,37.0,げん玉,ポイント,2020-04-27 15:56:21,2020-04-27 16:04:06,43922,155051921865339,BGT,5000,保留,保留


In [23]:
#シート名の確認
print(bk.sheet_names[0])

成果データ


「成果データ」シート分の対応

In [24]:
#余計なカラム削除
#df_dg = df_dg.drop(columns=['Unnamed: 9','成果'])
df_dg = df_dg.drop(columns=['成果','カード発行日'])

In [25]:
#ASIDがfloatになっていまうので整数化
for i in range(len(df_dg)):
    if pd.isnull(df_dg['ASID'][i]) == True:
        df_dg.loc[:,'ASID'][i] = 0
df_dg = df_dg.astype({'ASID':int})

print(df_dg.shape)
print(df_dg.index)
print(df_dg.columns)
print(df_dg.dtypes)

(821, 9)
RangeIndex(start=0, stop=821, step=1)
Index(['ASID', 'メディア名', '媒体チャネル', 'クリック', '申込', '申込月', '参照用', 'ASP名', '単価'], dtype='object')
ASID               int32
メディア名             object
媒体チャネル            object
クリック              object
申込        datetime64[ns]
申込月                int64
参照用                int64
ASP名              object
単価                 int64
dtype: object


C:\Users\crie072\anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [26]:
#"申込年月"のカラム生成
df_dg['申込年'] = df_dg['申込'].dt.year
df_dg['申込月'] = df_dg['申込'].dt.month

df_dg['申込年月'] = df_dg[['申込年', '申込月']].apply(lambda x: '{}年{}月'.format(x[0], x[1]), axis=1)
df_dg = df_dg.drop(columns=['申込年','申込月'])

df_dg.head()

,ASID,メディア名,媒体チャネル,クリック,申込,参照用,ASP名,単価,申込年月
0,8163,POM[ポム],ポイント,2020-04-23 10:31:22,2020-04-23 10:43:06,155051921858227,ADV,5000,2020年4月
1,8163,POM[ポム],ポイント,2020-04-24 10:17:31,2020-04-24 10:27:05,155051921859886,ADV,5000,2020年4月
2,37,げん玉,ポイント,2020-04-26 20:21:03,2020-04-26 20:29:07,155051921864571,BGT,5000,2020年4月
3,1580002,クレジットカード忍法帖,比較,2020-04-27 13:43:00,2020-04-27 14:31:00,155051921865248,JA,9000,2020年4月
4,37,げん玉,ポイント,2020-04-27 15:56:21,2020-04-27 16:04:06,155051921865339,BGT,5000,2020年4月


デジガレデータに承認データの"成果"をマージ

In [27]:
#承認データ（df_app）をマージする前に、df_app余計なカラムを削除
df_app_tmp = df_app.drop(columns=['入会申込番号'])

#承認データの"成果"をマージ
df_dg2 = pd.merge(df_dg,df_app_tmp, on =['参照用'], how = 'left')
df_dg2[df_dg2['成果'] == '可'].head()

,ASID,メディア名,媒体チャネル,クリック,申込,参照用,ASP名,単価,申込年月,成果,カード加入日
148,619,モッピー（連携用）,ポイント,2020-05-29 11:22:59,2020-05-29 11:35:05,155051921900938,BGT,7000,2020年5月,可,20200814.0
168,49,ハピタス（連携用）,ポイント,2020-05-31 16:39:35,2020-05-31 17:19:11,155051921904187,BGT,7000,2020年5月,可,20200813.0
201,619,モッピー（連携用）,ポイント,2020-06-06 22:37:01,2020-06-06 22:50:07,155051921910549,BGT,7000,2020年6月,可,20200730.0
223,619,モッピー（連携用）,ポイント,2020-06-10 12:03:52,2020-06-10 12:14:06,155051921914533,BGT,7000,2020年6月,可,20200722.0
231,619,モッピー（連携用）,ポイント,2020-06-12 00:10:32,2020-06-12 00:35:06,155051921916363,BGT,7000,2020年6月,可,20200803.0


In [28]:
df_dg2 = df_dg2.copy()

#マージしたらカード加入日がfloat型になってしまったため、整数型に戻す
for i in range(len(df_dg2)):
    if pd.isnull(df_dg2['カード加入日'][i]) == True:
        df_dg2.loc[:,'カード加入日'][i] = 0
        
df_dg2 = df_dg2.astype({'カード加入日':int})

C:\Users\crie072\anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [29]:
df_dg2[df_dg2['カード加入日'] == 0].head()

,ASID,メディア名,媒体チャネル,クリック,申込,参照用,ASP名,単価,申込年月,成果,カード加入日
0,8163,POM[ポム],ポイント,2020-04-23 10:31:22,2020-04-23 10:43:06,155051921858227,ADV,5000,2020年4月,NaN,0
1,8163,POM[ポム],ポイント,2020-04-24 10:17:31,2020-04-24 10:27:05,155051921859886,ADV,5000,2020年4月,NaN,0
2,37,げん玉,ポイント,2020-04-26 20:21:03,2020-04-26 20:29:07,155051921864571,BGT,5000,2020年4月,NaN,0
3,1580002,クレジットカード忍法帖,比較,2020-04-27 13:43:00,2020-04-27 14:31:00,155051921865248,JA,9000,2020年4月,NaN,0
4,37,げん玉,ポイント,2020-04-27 15:56:21,2020-04-27 16:04:06,155051921865339,BGT,5000,2020年4月,NaN,0


In [30]:
#"成果"がNanのものは[保留]に置換する
for i in range(len(df_dg2)):
    if pd.isnull(df_dg2['成果'][i]) == True:
        df_dg2.loc[:,'成果'][i] = "否認"#保留から変更 20/8/25
        

#カード加入日が0のものは[保留]に置換する
for i in range(len(df_dg2)):
    if df_dg2['カード加入日'][i] == 0:
        df_dg2.loc[:,'カード加入日'][i] = "否認"#保留から変更 20/8/25

C:\Users\crie072\anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
C:\Users\crie072\anaconda3\lib\site-packages\ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.
C:\Users\crie072\anaconda3\lib\site-packages\pandas\core\indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-cop

In [31]:
df_dg2.head()

,ASID,メディア名,媒体チャネル,クリック,申込,参照用,ASP名,単価,申込年月,成果,カード加入日
0,8163,POM[ポム],ポイント,2020-04-23 10:31:22,2020-04-23 10:43:06,155051921858227,ADV,5000,2020年4月,否認,否認
1,8163,POM[ポム],ポイント,2020-04-24 10:17:31,2020-04-24 10:27:05,155051921859886,ADV,5000,2020年4月,否認,否認
2,37,げん玉,ポイント,2020-04-26 20:21:03,2020-04-26 20:29:07,155051921864571,BGT,5000,2020年4月,否認,否認
3,1580002,クレジットカード忍法帖,比較,2020-04-27 13:43:00,2020-04-27 14:31:00,155051921865248,JA,9000,2020年4月,否認,否認
4,37,げん玉,ポイント,2020-04-27 15:56:21,2020-04-27 16:04:06,155051921865339,BGT,5000,2020年4月,否認,否認


集計

In [32]:
#客数
df_dg2[df_dg2['成果'] == '可'].shape[0]

294

In [33]:
pd.crosstab(df_dg2['申込年月'],df_dg2['成果'])

成果,可,否認
申込年月,,
2020年1月,0,3
2020年2月,0,2
2020年4月,0,12
2020年5月,7,159
2020年6月,32,122
2020年7月,189,126
2020年8月,66,103


CSVアウトプット②

In [34]:
df_dg2.to_csv(data_dir /r"output"/f"{output_file_2}_{bk.sheet_names[0]}.csv", index=False,header=True,encoding='shift-jis')

利用条件による成果判定チェックは別ノートブックで
<br>ここでは単純に加入有無だけを確認

In [43]:
print(bk.sheet_names[2])

成果データ（5月～利用条件追加媒体）


③ デジガレ「利用条件」リスト

In [44]:
print(input_file_3)

df_dg = pd.read_excel(data_dir/r"input"/f"{input_file_3}", sheet_name = 2)
df_dg.head()

20200821〆【JFRカード御中】成果データ&問い合わせ.xlsx


,ASID,メディア名,媒体チャネル,クリック,申込,参照用,ASP名,単価,成果,カード発行日
0,37.0,げん玉,ポイント,2020-05-01 00:06:29,2020-05-01 00:22:04,155051921869356,BGT,9000,保留,保留
1,37.0,げん玉,ポイント,2020-05-01 01:21:40,2020-05-01 01:28:04,155051921869380,BGT,9000,保留,保留
2,37.0,げん玉,ポイント,2020-05-01 06:27:16,2020-05-01 06:34:04,155051921869422,BGT,9000,保留,保留
3,37.0,げん玉,ポイント,2020-05-01 07:46:58,2020-05-01 08:02:04,155051921869489,BGT,9000,保留,保留
4,5954.0,GetMoney!(JIPC参加企業)（連携用）,ポイント,2020-05-01 11:33:44,2020-05-01 11:46:06,155051921869679,BGT,9000,保留,保留


In [45]:
#余計なカラム削除
#df_dg = df_dg.drop(columns=['Unnamed: 9','成果'])
df_dg = df_dg.drop(columns=['成果','カード発行日'])
df_dg.head()

,ASID,メディア名,媒体チャネル,クリック,申込,参照用,ASP名,単価
0,37.0,げん玉,ポイント,2020-05-01 00:06:29,2020-05-01 00:22:04,155051921869356,BGT,9000
1,37.0,げん玉,ポイント,2020-05-01 01:21:40,2020-05-01 01:28:04,155051921869380,BGT,9000
2,37.0,げん玉,ポイント,2020-05-01 06:27:16,2020-05-01 06:34:04,155051921869422,BGT,9000
3,37.0,げん玉,ポイント,2020-05-01 07:46:58,2020-05-01 08:02:04,155051921869489,BGT,9000
4,5954.0,GetMoney!(JIPC参加企業)（連携用）,ポイント,2020-05-01 11:33:44,2020-05-01 11:46:06,155051921869679,BGT,9000


In [46]:
#ASIDの整数化
for i in range(len(df_dg)):
    if pd.isnull(df_dg['ASID'][i]) == True:
        df_dg.loc[:,'ASID'][i] = 0
df_dg = df_dg.astype({'ASID':int})

print(df_dg.shape)
print(df_dg.index)
print(df_dg.columns)
print(df_dg.dtypes)

(1000, 8)
RangeIndex(start=0, stop=1000, step=1)
Index(['ASID', 'メディア名', '媒体チャネル', 'クリック', '申込', '参照用', 'ASP名', '単価'], dtype='object')
ASID               int32
メディア名             object
媒体チャネル            object
クリック              object
申込        datetime64[ns]
参照用                int64
ASP名              object
単価                 int64
dtype: object


C:\Users\crie072\anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [47]:
#"申込年月"のカラム生成
df_dg['申込年'] = df_dg['申込'].dt.year
df_dg['申込月'] = df_dg['申込'].dt.month

df_dg['申込年月'] = df_dg[['申込年', '申込月']].apply(lambda x: '{}年{}月'.format(x[0], x[1]), axis=1)
df_dg = df_dg.drop(columns=['申込年','申込月'])

df_dg.head()

,ASID,メディア名,媒体チャネル,クリック,申込,参照用,ASP名,単価,申込年月
0,37,げん玉,ポイント,2020-05-01 00:06:29,2020-05-01 00:22:04,155051921869356,BGT,9000,2020年5月
1,37,げん玉,ポイント,2020-05-01 01:21:40,2020-05-01 01:28:04,155051921869380,BGT,9000,2020年5月
2,37,げん玉,ポイント,2020-05-01 06:27:16,2020-05-01 06:34:04,155051921869422,BGT,9000,2020年5月
3,37,げん玉,ポイント,2020-05-01 07:46:58,2020-05-01 08:02:04,155051921869489,BGT,9000,2020年5月
4,5954,GetMoney!(JIPC参加企業)（連携用）,ポイント,2020-05-01 11:33:44,2020-05-01 11:46:06,155051921869679,BGT,9000,2020年5月


In [48]:
#承認データ（df_app）をマージする前に、df_app余計なカラムを削除
df_app_tmp = df_app.drop(columns=['入会申込番号'])

#承認データの"成果"をマージ
df_dg2 = pd.merge(df_dg,df_app_tmp, on =['参照用'], how = 'left')
df_dg2[df_dg2['成果'] == '可'].head()

,ASID,メディア名,媒体チャネル,クリック,申込,参照用,ASP名,単価,申込年月,成果,カード加入日
29,37,げん玉,ポイント,2020-05-03 23:43:50,2020-05-03 23:53:09,155051921874067,BGT,9000,2020年5月,可,20200710.0
37,5954,GetMoney!(JIPC参加企業)（連携用）,ポイント,2020-05-04 23:42:09,2020-05-04 23:52:43,155051921875536,BGT,9000,2020年5月,可,20200622.0
39,21,ポイントインカム,ポイント,2020-05-05 11:27:57,2020-05-05 11:55:08,155051921876005,BGT,9000,2020年5月,可,20200629.0
41,82,ちょびリッチ。ドットコム,ポイント,2020-05-05 17:43:16,2020-05-05 17:51:09,155051921876633,BGT,9000,2020年5月,可,20200707.0
47,57,ライフメディア,ポイント,2020-05-06 13:11:20,2020-05-06 13:28:09,155051921877672,BGT,9000,2020年5月,可,20200625.0


In [49]:
df_dg2 = df_dg2.copy()

In [50]:
#マージしたらカード加入日がfloat型になってしまったため、整数型に戻す
for i in range(len(df_dg2)):
    if pd.isnull(df_dg2['カード加入日'][i]) == True:
        df_dg2.loc[:,'カード加入日'][i] = 0
        
df_dg2 = df_dg2.astype({'カード加入日':int})

df_dg2[df_dg2['カード加入日'] == 0].head()

C:\Users\crie072\anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


,ASID,メディア名,媒体チャネル,クリック,申込,参照用,ASP名,単価,申込年月,成果,カード加入日
0,37,げん玉,ポイント,2020-05-01 00:06:29,2020-05-01 00:22:04,155051921869356,BGT,9000,2020年5月,NaN,0
1,37,げん玉,ポイント,2020-05-01 01:21:40,2020-05-01 01:28:04,155051921869380,BGT,9000,2020年5月,NaN,0
2,37,げん玉,ポイント,2020-05-01 06:27:16,2020-05-01 06:34:04,155051921869422,BGT,9000,2020年5月,NaN,0
3,37,げん玉,ポイント,2020-05-01 07:46:58,2020-05-01 08:02:04,155051921869489,BGT,9000,2020年5月,NaN,0
4,5954,GetMoney!(JIPC参加企業)（連携用）,ポイント,2020-05-01 11:33:44,2020-05-01 11:46:06,155051921869679,BGT,9000,2020年5月,NaN,0


In [51]:
#"成果"がNanのものは[保留]に置換する
for i in range(len(df_dg2)):
    if pd.isnull(df_dg2['成果'][i]) == True:
        df_dg2.loc[:,'成果'][i] = "保留"
        
#カード加入日が0のものは[保留]に置換する
for i in range(len(df_dg2)):
    if df_dg2['カード加入日'][i] == 0:
        df_dg2.loc[:,'カード加入日'][i] = "保留"

C:\Users\crie072\anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
C:\Users\crie072\anaconda3\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':
C:\Users\crie072\anaconda3\lib\site-packages\pandas\core\indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_ind

In [52]:
df_dg2.head()

,ASID,メディア名,媒体チャネル,クリック,申込,参照用,ASP名,単価,申込年月,成果,カード加入日
0,37,げん玉,ポイント,2020-05-01 00:06:29,2020-05-01 00:22:04,155051921869356,BGT,9000,2020年5月,保留,保留
1,37,げん玉,ポイント,2020-05-01 01:21:40,2020-05-01 01:28:04,155051921869380,BGT,9000,2020年5月,保留,保留
2,37,げん玉,ポイント,2020-05-01 06:27:16,2020-05-01 06:34:04,155051921869422,BGT,9000,2020年5月,保留,保留
3,37,げん玉,ポイント,2020-05-01 07:46:58,2020-05-01 08:02:04,155051921869489,BGT,9000,2020年5月,保留,保留
4,5954,GetMoney!(JIPC参加企業)（連携用）,ポイント,2020-05-01 11:33:44,2020-05-01 11:46:06,155051921869679,BGT,9000,2020年5月,保留,保留


集計

In [53]:
#客数
df_dg2[df_dg2['成果'] == '可'].shape[0]

641

In [54]:
pd.crosstab(df_dg2['申込年月'],df_dg2['成果'])

成果,保留,可
申込年月,,
2020年5月,82,15
2020年6月,79,104
2020年7月,92,435
2020年8月,106,87


CSVアウトプット

In [55]:
df_dg2.to_csv(data_dir /r"output"/f"{output_file_2}_{bk.sheet_names[2]}.csv", index=False,header=True,encoding='shift-jis')

利用条件による成果判定チェックは別ノートブックで（その２）
<br>ここでは単純に加入有無だけを確認

In [22]:
print(bk.sheet_names[3])

成果データ（5月～利用条件追加媒体）_承認返却済み


③ デジガレ「利用条件」リスト

In [24]:
print(input_file_3)

df_dg = pd.read_excel(data_dir/r"input"/f"{input_file_3}", sheet_name = 3)
df_dg.head()

20200821〆【JFRカード御中】成果データ&問い合わせ.xlsx


,ASID,メディア名,媒体チャネル,クリック,申込,参照用,ASP名,単価,成果,カード発行日,カード売上額合計
0,37,げん玉,ポイント,2020-05-01 00:31:25,2020-05-01 00:52:05,155051921869372,BGT,9000,保留,20200615,NaN
1,21,ポイントインカム,ポイント,2020-05-01 06:15:50,2020-05-01 06:25:05,155051921869414,BGT,9000,保留,20200611,NaN
2,21,ポイントインカム,ポイント,2020-05-01 06:51:10,2020-05-01 07:03:03,155051921869455,BGT,9000,保留,20200525,NaN
3,21,ポイントインカム,ポイント,2020-05-01 07:24:26,2020-05-01 07:35:04,155051921869471,BGT,9000,可,20200501,30000.0
4,21,ポイントインカム,ポイント,2020-05-01 07:53:11,2020-05-01 08:06:04,155051921869497,BGT,9000,保留,20200525,NaN


In [18]:
#余計なカラム削除
#df_dg = df_dg.drop(columns=['Unnamed: 9','成果'])
#df_dg = df_dg.drop(columns=['成果','カード発行日'])
#df_dg.head()

,ASID,メディア名,媒体チャネル,クリック,申込,参照用,ASP名,単価
0,37.0,げん玉,ポイント,2020-05-01 00:06:29,2020-05-01 00:22:04,155051921869356,BGT,9000
1,37.0,げん玉,ポイント,2020-05-01 01:21:40,2020-05-01 01:28:04,155051921869380,BGT,9000
2,37.0,げん玉,ポイント,2020-05-01 06:27:16,2020-05-01 06:34:04,155051921869422,BGT,9000
3,37.0,げん玉,ポイント,2020-05-01 07:46:58,2020-05-01 08:02:04,155051921869489,BGT,9000
4,5954.0,GetMoney!(JIPC参加企業)（連携用）,ポイント,2020-05-01 11:33:44,2020-05-01 11:46:06,155051921869679,BGT,9000


In [25]:
#ASIDの整数化
for i in range(len(df_dg)):
    if pd.isnull(df_dg['ASID'][i]) == True:
        df_dg.loc[:,'ASID'][i] = 0
df_dg = df_dg.astype({'ASID':int})

print(df_dg.shape)
print(df_dg.index)
print(df_dg.columns)
print(df_dg.dtypes)

(364, 11)
RangeIndex(start=0, stop=364, step=1)
Index(['ASID', 'メディア名', '媒体チャネル', 'クリック', '申込', '参照用', 'ASP名', '単価', '成果',
       'カード発行日', 'カード売上額合計'],
      dtype='object')
ASID                 int32
メディア名               object
媒体チャネル              object
クリック        datetime64[ns]
申込          datetime64[ns]
参照用                  int64
ASP名                object
単価                   int64
成果                  object
カード発行日               int64
カード売上額合計           float64
dtype: object


In [26]:
#"申込年月"のカラム生成
df_dg['申込年'] = df_dg['申込'].dt.year
df_dg['申込月'] = df_dg['申込'].dt.month

df_dg['申込年月'] = df_dg[['申込年', '申込月']].apply(lambda x: '{}年{}月'.format(x[0], x[1]), axis=1)
df_dg = df_dg.drop(columns=['申込年','申込月'])

df_dg.head()

,ASID,メディア名,媒体チャネル,クリック,申込,参照用,ASP名,単価,成果,カード発行日,カード売上額合計,申込年月
0,37,げん玉,ポイント,2020-05-01 00:31:25,2020-05-01 00:52:05,155051921869372,BGT,9000,保留,20200615,NaN,2020年5月
1,21,ポイントインカム,ポイント,2020-05-01 06:15:50,2020-05-01 06:25:05,155051921869414,BGT,9000,保留,20200611,NaN,2020年5月
2,21,ポイントインカム,ポイント,2020-05-01 06:51:10,2020-05-01 07:03:03,155051921869455,BGT,9000,保留,20200525,NaN,2020年5月
3,21,ポイントインカム,ポイント,2020-05-01 07:24:26,2020-05-01 07:35:04,155051921869471,BGT,9000,可,20200501,30000.0,2020年5月
4,21,ポイントインカム,ポイント,2020-05-01 07:53:11,2020-05-01 08:06:04,155051921869497,BGT,9000,保留,20200525,NaN,2020年5月


In [91]:
#承認データ（df_app）をマージする前に、df_app余計なカラムを削除
df_app_tmp = df_app.drop(columns=['入会申込番号'])

#承認データの"成果"をマージ
df_dg2 = pd.merge(df_dg,df_app_tmp, on =['参照用'], how = 'left')
df_dg2[df_dg2['成果'] == '可'].head()

,ASID,メディア名,媒体チャネル,クリック,申込,参照用,ASP名,単価,申込年月,成果,カード加入日
22,37,げん玉,ポイント,2020-05-03 23:43:50,2020-05-03 23:53:09,155051921874067,BGT,9000,2020年5月,可,20200710.0
30,5954,GetMoney!(JIPC参加企業)（連携用）,ポイント,2020-05-04 23:42:09,2020-05-04 23:52:43,155051921875536,BGT,9000,2020年5月,可,20200622.0
32,21,ポイントインカム,ポイント,2020-05-05 11:27:57,2020-05-05 11:55:08,155051921876005,BGT,9000,2020年5月,可,20200629.0
34,82,ちょびリッチ。ドットコム,ポイント,2020-05-05 17:43:16,2020-05-05 17:51:09,155051921876633,BGT,9000,2020年5月,可,20200707.0
40,57,ライフメディア,ポイント,2020-05-06 13:11:20,2020-05-06 13:28:09,155051921877672,BGT,9000,2020年5月,可,20200625.0


In [92]:
df_dg2 = df_dg2.copy()

In [93]:
#マージしたらカード加入日がfloat型になってしまったため、整数型に戻す
for i in range(len(df_dg2)):
    if pd.isnull(df_dg2['カード加入日'][i]) == True:
        df_dg2.loc[:,'カード加入日'][i] = 0
        
df_dg2 = df_dg2.astype({'カード加入日':int})

df_dg2[df_dg2['カード加入日'] == 0].head()

C:\Users\crie072\anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


,ASID,メディア名,媒体チャネル,クリック,申込,参照用,ASP名,単価,申込年月,成果,カード加入日
0,57,ライフメディア,ポイント,2020-05-02 01:00:13,2020-05-02 01:17:04,155051921870859,BGT,9000,2020年5月,NaN,0
1,37,げん玉,ポイント,2020-05-02 07:09:24,2020-05-02 07:21:05,155051921870958,BGT,9000,2020年5月,NaN,0
2,57,ライフメディア,ポイント,2020-05-02 06:56:25,2020-05-02 07:37:05,155051921870966,BGT,9000,2020年5月,NaN,0
3,57,ライフメディア,ポイント,2020-05-02 08:36:09,2020-05-02 08:55:04,155051921870990,BGT,9000,2020年5月,NaN,0
4,21,ポイントインカム,ポイント,2020-05-02 12:03:51,2020-05-02 12:18:04,155051921871345,BGT,9000,2020年5月,NaN,0


In [95]:
#"成果"がNanのものは[保留]に置換する
for i in range(len(df_dg2)):
    if pd.isnull(df_dg2['成果'][i]) == True:
        df_dg2.loc[:,'成果'][i] = "否認"#保留から変更 20/8/25
        
#カード加入日が0のものは[保留]に置換する
for i in range(len(df_dg2)):
    if df_dg2['カード加入日'][i] == 0:
        df_dg2.loc[:,'カード加入日'][i] = "否認"#保留から変更 20/8/25

In [58]:
df_dg2.head()

,ASID,メディア名,媒体チャネル,クリック,申込,参照用,ASP名,単価,申込年月,成果,カード加入日
0,37,げん玉,ポイント,2020-05-01 00:06:29,2020-05-01 00:22:04,155051921869356,BGT,9000,2020年5月,保留,保留
1,37,げん玉,ポイント,2020-05-01 01:21:40,2020-05-01 01:28:04,155051921869380,BGT,9000,2020年5月,保留,保留
2,37,げん玉,ポイント,2020-05-01 06:27:16,2020-05-01 06:34:04,155051921869422,BGT,9000,2020年5月,保留,保留
3,37,げん玉,ポイント,2020-05-01 07:46:58,2020-05-01 08:02:04,155051921869489,BGT,9000,2020年5月,保留,保留
4,5954,GetMoney!(JIPC参加企業)（連携用）,ポイント,2020-05-01 11:33:44,2020-05-01 11:46:06,155051921869679,BGT,9000,2020年5月,保留,保留


集計

In [96]:
#客数
df_dg2[df_dg2['成果'] == '可'].shape[0]

413

In [97]:
pd.crosstab(df_dg2['申込年月'],df_dg2['成果'])

成果,保留,可
申込年月,,
2020年5月,74,13
2020年6月,84,99
2020年7月,222,301


CSVアウトプット

In [62]:
df_dg2.to_csv(data_dir /r"output"/f"{output_file_2}_{bk.sheet_names[3]}.csv", index=False,header=True,encoding='shift-jis')